In [1]:
import gmaps
from api_keys import gkey
import gmaps.geojson_geometries
import requests
import json
from matplotlib.cm import Wistia
from matplotlib.colors import to_hex
import pandas as pd

gmaps.configure(api_key=gkey)

In [2]:
zip_geojson = requests.get('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tx_texas_zip_codes_geo.min.json').content
zip_geojson = json.loads(zip_geojson)

In [12]:
flooding_df = pd.read_csv('../data/rawdata/flooding_insurance_by_zip.csv')
print(len(flooding_df.index))
flooding_df.head()

130


,StateCode,CountyName,PostalCode,Locations,Building_TIV,Contents_TIV,Building_Limit,Contents_Limit
0,TX,Harris,77002,157,1.146599e+09,1.390003e+08,122273700,29113200
1,TX,Harris,77003,287,2.175743e+08,2.843671e+07,63492400,30093900
2,TX,Harris,77004,1052,8.208479e+08,7.126165e+07,335596500,92368400
3,TX,Harris,77005,4043,4.296319e+09,4.542693e+08,988278400,380594300
4,TX,Harris,77006,1318,7.685005e+08,8.814417e+07,314864400,127083900


In [16]:
building_numbers = pd.read_csv("../data/rawdata/Total_Addresses_Without_Apartments.csv", header = None)
print(len(building_numbers.index))
building_numbers = building_numbers[building_numbers[0].isin(list(flooding_df["PostalCode"]))]
building_numbers = building_numbers.reset_index(drop=True)
print(len(building_numbers.index))
building_numbers.head()

146
130


,0,1
0,77002,1010
1,77004,7161
2,77003,4036
3,77006,5672
4,77005,8192


In [6]:
zip_coords = pd.read_csv("../data/rawdata/zipcode_lat_lngs.csv")

In [8]:
have_zips = zip_coords[zip_coords["ZIP"].isin(list(flooding_df["PostalCode"]))]
have_zips = have_zips.reset_index(drop=True)
print(len(have_zips))
have_zips.head()

130


,ZIP,LAT,LNG
0,77002,29.756845,-95.365652
1,77003,29.749778,-95.345885
2,77004,29.724893,-95.363752
3,77005,29.718435,-95.423555
4,77006,29.740970,-95.391301


In [26]:
tot = pd.merge(flooding_df, building_numbers,left_on="PostalCode",right_on=0)
tot = tot.drop(0,axis=1)
tot = tot.merge(have_zips,left_on="PostalCode",right_on="ZIP")
tot = tot.drop("ZIP",axis=1)
tot = tot.rename(columns={1:"Total_Buildings"})
tot.head()

,StateCode,CountyName,PostalCode,Locations,Building_TIV,Contents_TIV,Building_Limit,Contents_Limit,Total_Buildings,LAT,LNG
0,TX,Harris,77002,157,1.146599e+09,1.390003e+08,122273700,29113200,1010,29.756845,-95.365652
1,TX,Harris,77003,287,2.175743e+08,2.843671e+07,63492400,30093900,4036,29.749778,-95.345885
2,TX,Harris,77004,1052,8.208479e+08,7.126165e+07,335596500,92368400,7161,29.724893,-95.363752
3,TX,Harris,77005,4043,4.296319e+09,4.542693e+08,988278400,380594300,8192,29.718435,-95.423555
4,TX,Harris,77006,1318,7.685005e+08,8.814417e+07,314864400,127083900,5672,29.740970,-95.391301


In [28]:
tot["Percent_Insured"] = tot["Locations"]/tot["Total_Buildings"]*100
tot.head()

,StateCode,CountyName,PostalCode,Locations,Building_TIV,Contents_TIV,Building_Limit,Contents_Limit,Total_Buildings,LAT,LNG,Percent_Insured
0,TX,Harris,77002,157,1.146599e+09,1.390003e+08,122273700,29113200,1010,29.756845,-95.365652,15.544554
1,TX,Harris,77003,287,2.175743e+08,2.843671e+07,63492400,30093900,4036,29.749778,-95.345885,7.111001
2,TX,Harris,77004,1052,8.208479e+08,7.126165e+07,335596500,92368400,7161,29.724893,-95.363752,14.690686
3,TX,Harris,77005,4043,4.296319e+09,4.542693e+08,988278400,380594300,8192,29.718435,-95.423555,49.353027
4,TX,Harris,77006,1318,7.685005e+08,8.814417e+07,314864400,127083900,5672,29.740970,-95.391301,23.236953


In [30]:
needed_zips = []
needed_zips = list(have_zips["ZIP"])

for i in needed_zips:
    i = str(i)

In [31]:
list_index = []

for i in range(len(zip_geojson['features'])):
    if zip_geojson['features'][i]['properties']['ZCTA5CE10'] not in str(needed_zips):
        list_index.append(i)
        
for index in sorted(list_index, reverse=True): 
    del zip_geojson['features'][index]
len(zip_geojson['features'])

130

In [32]:
sort_list = []
for i in range(len(zip_geojson["features"])):
    comp = int(zip_geojson['features'][i]['properties']['ZCTA5CE10'])
    sort_list.extend(tot.loc[tot["PostalCode"] == comp].index)
tot = tot.iloc[sort_list]
tot = tot.reset_index(drop=True)
tot.head()

,StateCode,CountyName,PostalCode,Locations,Building_TIV,Contents_TIV,Building_Limit,Contents_Limit,Total_Buildings,LAT,LNG,Percent_Insured
0,TX,Harris,77099,2369,8.183150e+08,8.967372e+07,395694800,98903600,8405,29.670869,-95.585990,28.185604
1,TX,Harris,77389,2001,1.592852e+09,1.702676e+08,457581700,182958800,7853,30.123469,-95.517064,25.480708
2,TX,Harris,77586,5560,3.457036e+09,3.666720e+08,1287637800,447320600,7425,29.583816,-95.036166,74.882155
3,TX,Harris,77020,246,7.120632e+07,9.499487e+06,46506300,24404700,7163,29.773179,-95.314327,3.434315
4,TX,Harris,77013,781,3.678831e+08,3.985653e+07,142362100,52088000,3420,29.795268,-95.233977,22.836257


In [41]:
#This top section is unused, ignore it. You still need to run the cell
# We will need to scale the GINI values to lie between 0 and 1
#min_gini = min(harris_zips)
#max_gini = max(harris_zips)
#gini_range = max_gini - min_gini

# def calculate_color(gini):
#     """
#     Convert the GINI coefficient to a color
#     """
#     # make gini a number between 0 and 1
#     normalized_gini = (gini - min_gini) / gini_range

#     # invert gini so that high inequality gives dark color
#     inverse_gini = 1.0 - normalized_gini

#     # transform the gini coefficient to a matplotlib color
#     mpl_color = viridis(inverse_gini)

#     # transform from a matplotlib color to a valid CSS color
#     gmaps_color = to_hex(mpl_color, keep_alpha=False)

#     return gmaps_color

def calculate_colors(input_list):
    _min = tot["Percent_Insured"].min()
    _max = tot["Percent_Insured"].max()
    _range = _max - _min
    
    result = []
    
    for i in input_list:
        norm = (i - _min) / _range
        
        inverse = 1.0 - norm
        mpl_color = Wistia(inverse)
        gmaps_color = to_hex(mpl_color, keep_alpha=False)
        
        result.append(gmaps_color)
        
    return result

def calculate_colors_dupe(input_list):
    _min = tot["Building_TIV"].min()
    _max = tot["Building_TIV"].max()
    _range = _max - _min
    
    result = []
    
    for i in input_list:
        norm = (i - _min) / _range
        
        inverse = 1.0 - norm
        mpl_color = Wistia(inverse)
        gmaps_color = to_hex(mpl_color, keep_alpha=False)
        
        result.append(gmaps_color)
        
    return result

In [42]:
colors = calculate_colors(tot["Percent_Insured"])

In [50]:
fig = gmaps.figure(map_type = "HYBRID")

gini_layer = gmaps.geojson_layer(
    zip_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
# zip_layer = gmaps.geojson_layer(zip_geojson)
#restaraunt_layer = gmaps.heatmap_layer(have_zips[['LAT', 'LNG']], weights = price["estimate"])
# fig.add_layer(restaraunt_layer)
# fig.add_layer(zip_layer)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
colors = calculate_colors_dupe(tot["Building_TIV"])

In [46]:
fig = gmaps.figure(map_type = "HYBRID")

gini_layer = gmaps.geojson_layer(
    zip_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
#zip_layer = gmaps.geojson_layer(zip_geojson)
#restaraunt_layer = gmaps.heatmap_layer(have_zips[['LAT', 'LNG']], weights = price["estimate"])
# fig.add_layer(restaraunt_layer)
#fig.add_layer(zip_layer)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))